In [ ]:
import pandas as pd
from pathlib import Path
import pickle

In [ ]:
upstream = ['read-csv']
product = None

In [ ]:
X_cols = pickle.loads(Path(upstream['read-csv']['X_cols']).read_bytes())
Y_col = pickle.loads(Path(upstream['read-csv']['Y_col']).read_bytes())
df = pickle.loads(Path(upstream['read-csv']['df']).read_bytes())

## Cleaning and converting into desired format

In [ ]:
# These names are required by the library
column_renamer = {"location": "unique_id",
                  "date": "ds",
                  "new_cases": "y"}

In [ ]:
# Library requires that all series end at the same time. Not many countries are lost this way
df = df[df.groupby("location")["date"].transform(max) == df["date"].max()]
minmaxes = df.groupby("location")["date"].agg([min, max, "count"])
minmaxes["proportions"] = (((minmaxes["max"] - minmaxes["min"]).dt.days + 1) / minmaxes["count"])
minmaxes[minmaxes["proportions"] != 1]

In [ ]:
# Aggregating data to be at a weekly level since daily was very messy
grouper_aggregator = {col: "mean" for col in X_cols}
grouper_aggregator.update({Y_col: "sum"})
df = df.groupby(["location", 
                 pd.Grouper(key='date', freq='W-MON')]).agg(grouper_aggregator).reset_index()

In [ ]:
Path(product['column_renamer']).parent.mkdir(exist_ok=True, parents=True)
Path(product['column_renamer']).write_bytes(pickle.dumps(column_renamer))

Path(product['df']).parent.mkdir(exist_ok=True, parents=True)
Path(product['df']).write_bytes(pickle.dumps(df))